In [ ]:
library(dplyr)
library(tidyr)

In [111]:
All_RBP_motifs <- read.csv("/home/ylee/AllmotifDB/All_RBP_motifs.csv")
rbp_list <- unique(All_RBP_motifs$RBP_name)

## Function

In [118]:
## alignment_details functions

alignment_details <- function(rbp_list, RBP_motifs, fimo_base_path) {
    motif_details <- data.frame(
        database = character(),
        RBP_name = character(),
        motif_id = character(),
        motif_alt_id = character(),
        alignment = logical(),
        start = numeric(),
        stop = numeric(),
        strand = character(),
        score = numeric(),
        p_value = numeric(),
        q_value = numeric(),
        matched_sequence = character(),
        stringsAsFactors = FALSE
    )
    
    for (rbp in rbp_list){
        fimo_file <- paste0(fimo_base_path, rbp, "/fimo.tsv")
        fimo_data <- read.delim(fimo_file, header = TRUE)

        motif_data <- RBP_motifs %>%
            filter(RBP_name == rbp) %>%
            select(database, RBP_name, motif_id, motif_alt_id)

        final_data <- list()
        for (i in 1:nrow(motif_data)) {
            if ("motif_id" %in% colnames(fimo_data) && "motif_alt_id" %in% colnames(fimo_data)){
                fimo_row <- fimo_data %>%  
                    filter(motif_id == motif_data[i, "motif_id"], motif_alt_id == motif_data[i, "motif_alt_id"])
                
                if (nrow(fimo_row) > 0) {
                    # If alignment data exists, fill in alignment data
                    alignment <- TRUE
                    start <- fimo_row$start
                    stop <- fimo_row$stop
                    strand <- fimo_row$strand
                    score <- fimo_row$score
                    p_value <- fimo_row$p.value
                    q_value <- fimo_row$q.value
                    matched_sequence <- fimo_row$matched_sequence
                } else {
                    # If no alignment data, set to NA
                    alignment <- FALSE
                    start <- NA
                    stop <- NA
                    strand <- NA
                    score <- NA
                    p_value <- NA
                    q_value <- NA
                    matched_sequence <- NA
                }
            }else {
                    # If no alignment data, set to NA
                    alignment <- FALSE
                    start <- NA
                    stop <- NA
                    strand <- NA
                    score <- NA
                    p_value <- NA
                    q_value <- NA
                    matched_sequence <- NA
            }

            
            # Create a data frame for the current motif and append to motif_details
            motif_details <- bind_rows(motif_details, data.frame(
                database = motif_data[i, "database"],
                RBP_name = motif_data[i, "RBP_name"],
                motif_id = motif_data[i, "motif_id"],
                motif_alt_id = motif_data[i, "motif_alt_id"],
                alignment = alignment,
                start = start,
                stop = stop,
                strand = strand,
                score = score,
                p_value = p_value,
                q_value = q_value,
                matched_sequence = matched_sequence,
                stringsAsFactors = FALSE
            ))
        }
    }
    return(motif_details)    
}

In [108]:
alignment_combined_details <- function(csv_paths, mapped_data, sequence_name) {
    alignment_combined <- data.frame(
        fragment_id = character(),
        database = character(),
        RBP_name = character(),
        motif_id = character(),
        motif_alt_id = character(),
        alignment = logical(),
        start = numeric(),
        stop = numeric(),
        strand = character(),
        score = numeric(),
        p_value = numeric(),
        q_value = numeric(),
        matched_sequence = character(),
        stringsAsFactors = FALSE
    )
    
    mapped_data[, 1] <- paste0(sequence_name, "_", mapped_data[, 1] + 1)
    colnames(mapped_data)[1] <- "fragment_id"
    mapped_data$motif_alignments <- NA
    
    for (csv_path in csv_paths){
        fragment_number <- as.numeric(sub(paste0(".*", sequence_name, "_(\\d+)\\.csv$"), "\\1", csv_path))
        fragment_start <- mapped_data[fragment_number, "position.in.adapting.gene.DNA"]
        
        alignment_df <- read.csv(csv_path)
        filtered_df <- alignment_df[alignment_df$alignment == TRUE, ]
        mapped_data$motif_alignments[fragment_number] <- nrow(filtered_df)
        
        if (nrow(filtered_df) > 0) {
            filtered_df$fragment_id <- paste0(sequence_name, "_", fragment_number)
            filtered_df$start <- filtered_df$start + fragment_start
            filtered_df$stop <- filtered_df$stop + fragment_start
            alignment_combined <- bind_rows(alignment_combined, filtered_df)
        }
    }

    return(list(alignment_combined = alignment_combined, mapped_data = mapped_data))
}

## Combine Motif Scanning Result - Actg1

In [119]:
sequence_name = "Actg1_hotspot"                            ### Change Here : Actg1_hotspot, Actg1_ctrl, Actg1_random
base_dir <- paste0("/home/ylee/meme/fimo_out/", sequence_name)
csv_dir <- paste0(base_dir, "/alignment_csv")

fimo_base_path <- paste0(base_dir, "/")
dir.create(csv_dir)

motif_details <- alignment_details(rbp_list, All_RBP_motifs, fimo_base_path)

output_dir <- paste0(csv_dir, "/", basename(fimo_base_path), ".csv")
write.csv(motif_details, output_dir, row.names = FALSE) 

## Combine Motif Scanning Result - FERMT1/TPM2

In [ ]:
sequence_name = "FERMT1"                                      ### Change Here : FERMT1, TPM2
base_dir <- paste0("/home/ylee/meme/fimo_out/", sequence_name)
csv_dir <- paste0(base_dir, "/alignment_csv")

fimo_base_paths <- paste0(list.dirs(base_dir, recursive = FALSE, full.names = TRUE), "/")
dir.create(csv_dir)

for (fimo_base_path in fimo_base_paths){
    motif_details <- alignment_details(rbp_list, All_RBP_motifs, fimo_base_path)    
    
    output_dir <- paste0(csv_dir, "/", basename(fimo_base_path), ".csv")
    print(output_dir)
    flush.console()
    write.csv(motif_details, output_dir, row.names = FALSE)   
}

In [138]:
if (sequence_name == "FERMT1"){
    mapped_data <- read.csv("/home/ylee/meme/FERMT1_TPM2_sequence/PANX1_NB_fragment_FERMT1_mapped_sequecnes_64_percent.csv", header = TRUE, fill = TRUE)
} else {     
    mapped_data <- read.csv("/home/ylee/meme/FERMT1_TPM2_sequence/PANX1_NB_fragment_TPM2_mapped_sequecnes_64_percent.csv", header = TRUE, fill = TRUE)
}

In [139]:
csv_paths <- list.files(csv_dir, recursive = FALSE, full.names = TRUE)
csv_paths <- csv_paths[grepl(paste0("^", sequence_name, "_\\d+\\.csv$"), basename(csv_paths))]

results <- alignment_combined_details(csv_paths, mapped_data, sequence_name)
write.csv(results$mapped_data, paste0(csv_dir, "/", sequence_name, "_mapped_motif_alignments_summary.csv"), row.names = FALSE) 
write.csv(results$alignment_combined, paste0(csv_dir, "/", sequence_name, "_mapped_motif_alignments.csv"), row.names = FALSE)  

## Combine Motif Scanning Result-Actg2

In [97]:
sequence_name = "Actg2_ctrl"                                ### Change Here : Actg2_hotspot, Actg2_ctrl, Actg2_random
base_dir <- paste0("/home/ylee/meme/fimo_out/", sequence_name)
csv_dir <- paste0(base_dir, "/alignment_csv")

fimo_base_paths <- paste0(list.dirs(base_dir, recursive = FALSE, full.names = TRUE), "/")
dir.create(csv_dir)

for (fimo_base_path in fimo_base_paths){
    motif_details <- alignment_details(rbp_list, All_RBP_motifs, fimo_base_path)    
    
    output_dir <- paste0(csv_dir, "/", basename(fimo_base_path), ".csv")
    write.csv(motif_details, output_dir, row.names = FALSE)   
}

   database RBP_name motif_id motif_alt_id
1 CisBP_RNA     FXR2     1.10      DGACRRR
  motif_id motif_alt_id       sequence_name start stop strand  score p.value
1     1.10      DGACRRR Actg2_ctrl_1_P16368     6   12      + 10.897   8e-05
  q.value matched_sequence
1 0.00304          GGACGAG
  database RBP_name motif_id motif_alt_id
2 oRNAment     FXR2      1.2      DGACRRR
  motif_id motif_alt_id       sequence_name start stop strand  score p.value
1      1.2      DGACRRR Actg2_ctrl_1_P16368     6   12      + 10.897   8e-05
  q.value matched_sequence
1 0.00304          GGACGAG
  database RBP_name motif_id motif_alt_id
1   mCross     ILF3    1.5.1  BKSAGCCTCNB
  motif_id motif_alt_id       sequence_name start stop strand   score  p.value
1    1.5.1  BKSAGCCTCNB Actg2_ctrl_1_P16368     8   18      - 14.2879 4.53e-06
   q.value matched_sequence
1 0.000136      CTGAGCCTCGT
  database RBP_name motif_id motif_alt_id
3 oRNAment    MBNL1    1.1.2      GCWTGCN
  motif_id motif_alt_id       se

In [100]:
if (sequence_name == "Actg2_hotspot"){
    mapped_data <- read.csv("/home/ylee/meme/Actg2_sequence/Actg1_hotspot101_Actg2_mapped_sequecnes_68_percent.csv", header = TRUE, fill = TRUE)
} else if (sequence_name == "Actg2_ctrl"){
    mapped_data <- read.csv("/home/ylee/meme/Actg2_sequence/Actg1_ctrl_Actg2_mapped_sequecnes_68_percent.csv", header = TRUE, fill = TRUE)
} else {     
    mapped_data <- read.csv("/home/ylee/meme/Actg2_sequence/Actg1_random_Actg2_mapped_sequecnes_68_percent.csv", header = TRUE, fill = TRUE)
}

In [109]:
csv_paths <- list.files(csv_dir, recursive = FALSE, full.names = TRUE)
csv_paths <- csv_paths[grepl(paste0("^", sequence_name, "_\\d+\\.csv$"), basename(csv_paths))]

results <- alignment_combined_details(csv_paths, mapped_data, sequence_name)
write.csv(results$mapped_data, paste0(csv_dir, "/", sequence_name, "_mapped_motif_alignments_summary.csv"), row.names = FALSE) 
write.csv(results$alignment_combined, paste0(csv_dir, "/", sequence_name, "_mapped_motif_alignments.csv"), row.names = FALSE)  